# Cross region model copy

**Context:**
By default, models are only available in the region and account in which they were created. With model copy feature, you have the ability to copy models to other regions. You can copy `custom model` and `shared model` to other regions.

You can copy models to be used in supported regions. Check [supported region](https://docs.aws.amazon.com/bedrock/latest/userguide/copy-model-support.html) to find currently supported regions.

**How it works?**
1. Using Bedrock Console:
After logging on to your AWS account, navigate to `custom model` and then choose the model that was `shared` with you or have have `customized`. Selctec the three dots on the right hand side top (as shown in the image below) and click `copy`. You will now specify the region where you want to copy the model and the name you want to use for the copied model. Click `copy model` at the bottom to create a copy job.

2. Using API:
Find the ARN of the model you want to copy and then initiate a client in the region where you want to copy the model (this is your `target` region). To create a copy job use `create_model_copy_job` api using your target region client. Specify the name you want to use for the model in your target region and execute the api call. 

In this notebook, we wll use API to create a model copy job. Additonally, if you want to find the arn of the customized model using api, you can use `list_custom_models` api. 

In [ ]:
# Update the boto3 sdk to latest version.
! pip install boto3 --upgrade

In [ ]:
# restart the kernel:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3

### Find custom model arn
This will be used in `create_model_copy_job` api and create a copy model job.

In [ ]:
source_region_bedrock_client = boto3.client('bedrock', region_name='us-east-1')
list_custom_models = source_region_bedrock_client.list_custom_models()

In [ ]:
print(list_custom_models['modelSummaries'])

In [ ]:
# select the model arn you want to use:
model_arn = '<custom model arn to be copied to target region>'
target_model_name = '<name of the copied model in target region>'
custom_tag_key = '<your tag key>'
custom_tag_value = '<your tag value>'

### Create target region client and copy model job

In [ ]:
target_region_bedrock_client = boto3.client('bedrock', region_name='us-west-2')

In [ ]:
response = target_region_bedrock_client.create_model_copy_job(
    sourceModelArn=model_arn,
    targetModelName=target_model_name,
    targetModelTags=[
        {
            'key': custom_tag_key,
            'value': custom_tag_value
        },
    ]
)

print(response)

### Check status of model copy job

In [ ]:
target_region_bedrock_client.get_model_copy_job(
    jobArn=response['jobArn']
)

# End